In [17]:
import pandas as pd

# Load your data (adjust the path as needed)
wapo_data = pd.read_csv('../benchmark_data/fatal-police-shootings-data.csv')

# Define common suffixes to ignore
common_suffixes = ['Jr', 'Sr', 'II', 'III', 'IV', 'V']

# Define a function that processes each name
def process_name(name):
    # Ensure name is a string and not NaN or any float value
    if pd.isnull(name):
        # Handle NaN values or any other non-string values
        return [None, None]  # Or return ['Unknown', 'Unknown'] based on your preference
    else:
        # Convert to string in case it's not (handles numeric values)
        name = str(name)
        # Split the name into parts
        parts = name.split()
        # Check and construct the name ignoring common suffixes and middle names
        if parts[-1] in common_suffixes and len(parts) > 2:
            first_last_name = f"{parts[0]} {parts[-2]}"
        elif len(parts) > 2:
            first_last_name = f"{parts[0]} {parts[-1]}"
        else:
            first_last_name = name
        # Return a list with the original name, the first-last name version, and the version ignoring suffixes
        return [name, first_last_name]

# Apply the function to the 'name' column and create a new column with the results
wapo_data['names'] = wapo_data['name'].apply(process_name)
wapo_data['date'] = pd.to_datetime(wapo_data['date'])
wapo_data = wapo_data[(wapo_data['date'] >= '2023-12-01') & (wapo_data['date'] <= '2023-01-04')]

result = pd.read_csv('../data_storage/2023-12-01 00/00/00_weaviate.csv')
result['publication_date'] = pd.to_datetime(result['publication_date'])




FileNotFoundError: [Errno 2] No such file or directory: '../data_storage/2023-12-01 00/00/00_weaviate.csv'

In [ ]:
# Assuming the preparation steps have been done as in your provided code

# Helper function to check conditions
def check_conditions(row, results_df):
    # Check if any name in 'names' appears in 'snippet'
    for name in row['names']:
        if pd.notnull(name) and any(name in snippet for snippet in results_df['snippet']):
            return True
    
    # Convert 'date' to datetime for comparison
    wapo_date = pd.to_datetime(row['date'])
    
    # Check if 'city' appears in 'snippet' and dates within 7 days
    city_condition = results_df.apply(lambda x: row['city'] in x['snippet'] and abs((wapo_date - x['publication_date']).days) <= 7, axis=1)
    if city_condition.any():
        return True
    
    # Check if 'county' appears in 'snippet' and dates within 5 days
    county_condition = results_df.apply(lambda x: row['county'] in x['snippet'] and abs((wapo_date - x['publication_date']).days) <= 5, axis=1)
    if county_condition.any():
        return True
    
    # If none of the conditions are met
    return False

# Apply the helper function to each row in wapo_data
wapo_data['condition_met'] = wapo_data.apply(check_conditions, results_df=result, axis=1)

# Print summary
true_count = wapo_data['condition_met'].sum()
total_rows = len(wapo_data)
print(f"True conditions: {true_count} out of {total_rows} rows.")


In [18]:
from InstructorEmbedding import INSTRUCTOR
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L6-cos-v5')
model_ins = INSTRUCTOR('hkunlp/instructor-large')
model.max_seq_length = 512
model_ins.max_seq_length = 512
instruction = "Represent the news articles about police-involved shooting for retrieval:"

sentence = 'test'

snippet_vector = model.encode(sentence).tolist()
snippet_vector_ins = model.encode([[instruction,sentence]]).tolist()
snippet_vector_ins = [item for sublist in snippet_vector_ins for item in sublist]

print("sentence transformer model: \n")
print(snippet_vector)
print("instructor model: \n")
print(snippet_vector_ins)


load INSTRUCTOR_Transformer
max_seq_length  512
sentence transformer model: 

[0.046727247536182404, 0.043664414435625076, -0.021435163915157318, -0.02754155918955803, -0.02544456720352173, -0.03043508715927601, -0.010790720582008362, 0.005107902456074953, -0.023517614230513573, -0.036522962152957916, -0.0704219862818718, -0.018469570204615593, -3.5950033634435385e-06, -0.009923654608428478, -0.058486513793468475, -0.07244995981454849, -0.012400773353874683, -0.09254519641399384, 0.023529261350631714, -0.04025384038686752, -0.006299762986600399, -0.05812370777130127, 0.021734081208705902, -0.054823216050863266, -0.04591784626245499, -0.03559869900345802, 0.0406176894903183, -0.01410397607833147, -0.007450509816408157, -0.018193740397691727, 0.05266158655285835, -0.018498092889785767, -0.052554793655872345, -0.02019062452018261, 0.0312875397503376, -0.01895030587911606, -0.06330179423093796, -0.03767998889088631, 0.03101617470383644, -0.03167155757546425, 0.010693609714508057, -0.046425